## Import libraries, set options

In [2]:
#import libraries
import datetime
import dateutil.relativedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections
from ggplot import *
import requests, zipfile, StringIO

In [3]:
#set options
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.precision', 2)
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
%matplotlib inline
#set figure size
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 16, 12 

## Load Data

In [5]:
#Data source: 
#Data dictionary: http://www.fec.gov/finance/disclosure/metadata/DataDictionaryContributionsbyIndividuals.shtml

#r = requests.get('ftp://ftp.fec.gov/FEC/2016/indiv16.zip')
#z = zipfile.ZipFile(StringIO.StringIO(r.content))
#dt = pandas.read_csv(z.open('itcont.txt'))

#dt = pd.read_csv("itcont.txt", delimiter = '|', low_memory=False)

KeyboardInterrupt: 

In [178]:
#get committees associated with candidate 
#Hillary candidate id = P00003392
#Trump candidate id = P80001571
cm = pd.read_csv('cm.txt', delimiter = '|')
labels = pd.read_csv('cm_header_file.csv')
cm.columns = labels.columns
cm_labels =  cm.loc[:,['CMTE_ID', 'CMTE_NM']]
committies = cm[cm['CAND_ID'].isin(['P00003392', 'P80001571'])]
committies.to_csv('candidate_committees.csv')
committies

,CMTE_ID,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_DSGN,CMTE_TP,CMTE_PTY_AFFILIATION,CMTE_FILING_FREQ,ORG_TP,CONNECTED_ORG_NM,CAND_ID
9977,C00570978,FUTURE OF AMERICAN LIVES MATTER,LITTLE MASON LESURE,40 RIVER RD APT 7-K,NaN,NEW YORK,NY,1.00e+04,U,N,NaN,A,NaN,NaN,P00003392
10440,C00575795,HILLARY FOR AMERICA,"VILLARREAL, JOSE H.",P.O. BOX 5256,NaN,NEW YORK,NY,1.02e+08,P,P,DEM,M,NaN,NaN,P00003392
10596,C00577395,PEOPLE IN COMMAND/PIC,TRACEY SAPP,3674 RUNNYMEDE BLVD,NaN,CLEVELAND HTS,OH,4.41e+04,U,U,NaN,Q,NaN,NaN,P00003392
10861,C00580100,"DONALD J. TRUMP FOR PRESIDENT, INC.","JOST, TIMOTHY",725 FIFTH AVENUE,NaN,NEW YORK,NY,1.00e+04,P,P,REP,M,NaN,TRUMP VICTORY,P80001571
14389,C00615674,DONALD AND THE PROHET,H JOHN ROGERS,801 SOUTHERN AVENUE NO 2,NaN,PITTSBURGH,PA,1.52e+04,U,U,NaN,Q,NaN,NaN,P80001571
14983,C00621672,VERMONT TRUMP DELEGATE FUND,JOHN M LINDLEY,99 STATE STREET,NaN,MONTPELIER,VT,5.60e+03,U,N,UNK,Q,NaN,NaN,P80001571


In [469]:
#set reader to read data in chunks 
labels = pd.read_csv('indiv_header_file.csv') #labels for individual contribution file 
reader = pd.read_csv("itcont.txt", delimiter = '|', low_memory=False, encoding = 'utf-8', dtype = 'str', iterator=True, chunksize=3000)

In [470]:
dt = reader.get_chunk(12)

In [472]:
dt.tail(3)

,C00004606,N,M4,P,15951124869,15,IND,"ARNOLD, ROBERT",MCPHERSON,KS,67460,SELF,OPTOMETRIST,03102015,1000,Unnamed: 15,SA11AI.20747,1002259,Unnamed: 18,Unnamed: 19,4041320151241796098
9,C00411116,N,M4,NaN,15970337985,15,IND,"BUGYIK, PAUL L.",RESEDA,CA,913355843,SPACEX,ENGINEER - RF,03272015,100,NaN,AB262279D360A48109AC,1002269,NaN,NaN,4041320151241796108
10,C00411116,N,M4,NaN,15970337986,15,IND,"DEN HERDER, DAVID",WASHINGTON,DC,200362207,SPACEX,COUNSEL - SENIOR,03272015,94,NaN,A6CAFD79581F747E8A73,1002269,NaN,NaN,4041320151241796109
11,C00411116,N,M4,NaN,15970337986,15,IND,"DUNN, MATHEW P.",WASHINGTON,DC,200023606,SPACEX,DIRECTOR,03272015,140,NaN,AB2932F0B587B42E9B92,1002269,NaN,NaN,4041320151241796110


In [473]:
dt.head(10)
dt.shape

(12, 21)

In [474]:
#process data 
dt.columns = labels.columns
dt.head(10)

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,C00004606,N,M4,P,15951124869,15,IND,"BICKLE, DON",HAYS,KS,67601,RETIRED,RETIRED,03302015,1000,NaN,SA11AI.20772,1002259,NaN,NaN,4041320151241796099
1,C00004606,N,M4,P,15951124869,15,IND,"ROSSMAN, RICHARD",OLATHE,KS,66051,CRAWFORD SALES COMPANY,BUSINESSMAN,03302015,250,NaN,SA11AI.20759,1002259,NaN,NaN,4041320151241796100
2,C00452383,N,M4,P,15951124897,15,IND,"LLEWELLYN, CHARLES",FREDERICK,MD,21704,NaN,NaN,03112015,500,NaN,SA11AI.25088,1002261,NaN,NaN,4041320151241796102
3,C00452383,N,M4,P,15951124897,15,IND,"TYNES, TIMOTHY MR.",VERO BEACH,FL,329600595,NaN,NaN,03022015,250,NaN,SA11AI.25074,1002261,NaN,NaN,4041320151241796103
4,C00452383,N,M4,P,15951124898,15,IND,"WIESTER, GEORGIA MRS.",BUELLTON,CA,934279421,NONE,HOMEMAKER,03022015,200,NaN,SA11AI.25072,1002261,NaN,NaN,4041320151241796104
5,C00452383,N,M4,NaN,15951124897,15,IND,"BIGBEE, PHYLLIS MRS.",BROKEN BOW,NE,688224214,NONE,HOMEMAKER,03232015,70,NaN,SA11AI.25089,1002261,NaN,NaN,4041320151241796101
6,C00411116,N,M4,P,15970337992,15,IND,"SADHWANI, ANAND",HAWTHORNE,CA,902506844,SPACEX,ANALYST - PROPULSION - LEAD,03302015,3000,NaN,AAEA0ACE31DD54D0F989,1002269,NaN,NaN,4041320151241796127
7,C00411116,N,M4,NaN,15970337985,15,IND,"BEDNAREK, STEPHANIE",WASHINGTON,DC,200051503,SPACEX,GOVERNMENT AFFAIRS - MANAGER,03272015,110,NaN,AC15372A86E414D999AF,1002269,NaN,NaN,4041320151241796106
8,C00411116,N,M4,NaN,15970337985,15,IND,"BRUNSON, RAEGAN",LOS ANGELES,CA,900341308,SPACEX,OFFICE MANAGER,03272015,70,NaN,A44B021C322C945DAB7A,1002269,NaN,NaN,4041320151241796107
9,C00411116,N,M4,NaN,15970337985,15,IND,"BUGYIK, PAUL L.",RESEDA,CA,913355843,SPACEX,ENGINEER - RF,03272015,100,NaN,AB262279D360A48109AC,1002269,NaN,NaN,4041320151241796108


In [475]:
#read in data only for Clinton, Trump candidate ids. 
#candidate id: Hillary Clinton, 'P00003392', 'P80001571'
#candidate committee id: HILLARY FOR AMERICA, C00575795; DONALD J. TRUMP FOR PRESIDENT, INC., C00580100
icontribs = []
for df in reader:
    df.columns = labels.columns
    df = pd.DataFrame(df)
    candidates = df[df['CMTE_ID'].isin(['C00575795', 'C00580100'])]
    icontribs.append(candidates)

In [476]:
#concatenate remaining df's. 
df_stacked = pd.concat([i for i in icontribs], ignore_index=True)  

In [477]:
len(df_stacked)

421708

In [479]:
dt = df_stacked
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421708 entries, 0 to 421707
Data columns (total 21 columns):
CMTE_ID            421708 non-null object
AMNDT_IND          421708 non-null object
RPT_TP             421708 non-null object
TRANSACTION_PGI    421708 non-null object
IMAGE_NUM          421708 non-null object
TRANSACTION_TP     421708 non-null object
ENTITY_TP          421708 non-null object
NAME               421708 non-null object
CITY               421490 non-null object
STATE              420241 non-null object
ZIP_CODE           420218 non-null object
EMPLOYER           306446 non-null object
OCCUPATION         408304 non-null object
TRANSACTION_DT     421708 non-null object
TRANSACTION_AMT    421708 non-null object
OTHER_ID           1516 non-null object
TRAN_ID            421708 non-null object
FILE_NUM           421708 non-null object
MEMO_CD            550 non-null object
MEMO_TEXT          2254 non-null object
SUB_ID             421708 non-null object
dtypes: object

In [715]:
#convert date to datetime 
dt['TRANSACTION_DT'] = pd.to_datetime(dt['TRANSACTION_DT'], format = '%m%d%Y')
dt['TRANSACTION_AMT'] = dt['TRANSACTION_AMT'].astype(float)
dt.head(3)
dt.tail(3)

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
421705,C00575795,N,M6,G,201606209018549896,15,IND,"METAXOTOS, MICHAEL",HAZLET,NJ,077302265,OPPORTUNITY KNOCKS,EXECUTIVE DIRECTOR,2016-05-29,25,NaN,C5301399,1079219,NaN,NaN,4062320161300218227
421706,C00575795,N,M6,P,201606209018549896,15,IND,"DOWD, COLLEEN",BRIGHTON,CO,806013430,PVMC,NURSE,2016-05-01,8,NaN,C4601299,1079219,NaN,NaN,4062320161300218228
421707,C00575795,N,M6,P,201606209018549896,15,IND,"WOMACK, ELAINE",MOUNTAIN VIEW,CA,940402083,SAN JOSE UNIFIED SCHOOL DISTRICT,TEACHER,2016-05-23,50,NaN,C5143349,1079219,NaN,NaN,4062320161300218229


In [716]:
#get committee names and merge with data on committee id
cm = pd.read_csv('cm.txt', delimiter = '|')
labels = pd.read_csv('cm_header_file.csv')
cm.columns = labels.columns
cm_labels =  cm.loc[:,['CMTE_ID', 'CMTE_NM']]
dt_merged = pd.merge(left=dt, right=cm_labels, how='left', on ='CMTE_ID')

In [717]:
#get candidate links to committees and merge with data on committee id
ccl = pd.read_csv('ccl.txt', delimiter = '|')
labels = pd.read_csv('ccl_header_file.csv')
ccl.columns = labels.columns
ccl_labels =  ccl.loc[:,['CMTE_ID', 'CAND_ID', 'CMTE_TP']]
dt_merged2 = pd.merge(left=dt_merged, right=ccl_labels, how='left', on ='CMTE_ID')

In [718]:
#get candidate names and merge with data on candidate id
cn = pd.read_csv('cn.txt', delimiter = '|')
labels = pd.read_csv('cn_header_file.csv')
cn.columns = labels.columns
cn_labels =  cn.loc[:,['CAND_ID', 'CAND_NAME', 'CAND_PTY_AFFILIATION', 'CAND_OFFICE']]
dt_merged3 = pd.merge(left=dt_merged2, right=cn_labels, how='left', on ='CAND_ID')
dt_merged3

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CMTE_NM,CAND_ID,CMTE_TP,CAND_NAME,CAND_PTY_AFFILIATION,CAND_OFFICE
0,C00580100,N,Q2,P,201507159000184788,15,IND,"ANDALORO, JIM",MARBLEHEAD,MA,01945,"METAL TRONICS, INC.",OWNER,2015-06-30,500,NaN,SA17A.5601,1015464,NaN,NaN,4071620151247220745,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,P,"TRUMP, DONALD J.",REP,P
1,C00580100,N,Q2,P,201507159000184788,15,IND,"ANTON, FRANCINE",SPRING LAKE,MI,49456,RETIRED,RETIRED,2015-06-24,250,NaN,SA17A.5743,1015464,NaN,NaN,4071620151247220746,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,P,"TRUMP, DONALD J.",REP,P
2,C00580100,N,Q2,P,201507159000184788,15,IND,"ATKINS, LORI",SAN ANTONIO,TX,78231,TEXAS SINUS CENTER,PHYSICIAN ASSISTANT,2015-06-20,250,NaN,SA17A.4275,1015464,NaN,NaN,4071620151247220747,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,P,"TRUMP, DONALD J.",REP,P
3,C00580100,N,Q2,P,201507159000184789,15,IND,"BINGHAM, RON",JACKSON,TN,38305,EMG CLINIS OF TENNESSEE,MD,2015-06-19,500,NaN,SA17A.4383,1015464,NaN,NaN,4071620151247220748,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,P,"TRUMP, DONALD J.",REP,P
4,C00580100,N,Q2,P,201507159000184789,15,IND,"BLUE, JAN",HUNTINGTON BEACH,CA,92648,HOAG HEALTH,SENIOR VICE PRESIDENT,2015-06-17,500,NaN,SA17A.4155,1015464,NaN,NaN,4071620151247220749,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,P,"TRUMP, DONALD J.",REP,P
5,C00580100,N,Q2,P,201507159000184789,15,IND,"BRIDGES, PAMELA",LAUREL,MS,39443,INTERNATIONAL FIRE,PRESIDENT,2015-06-26,250,NaN,SA17A.5348,1015464,NaN,NaN,4071620151247220750,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,P,"TRUMP, DONALD J.",REP,P
6,C00580100,N,Q2,P,201507159000184790,15,IND,"BYRNES, JECOAH",COLORADO SPRINGS,CO,80920,"EMBREE CAPITAL MARKETS GROUP, INC.",SALES,2015-06-16,250,NaN,SA17A.5259,1015464,NaN,NaN,4071620151247220751,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,P,"TRUMP, DONALD J.",REP,P
7,C00580100,N,Q2,P,201507159000184790,15,IND,"CANNON, WANDA",BEVERLY HILLS,CA,90210,RETIRED,RETIRED,2015-06-16,250,NaN,SA17A.4913,1015464,NaN,NaN,4071620151247220752,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,P,"TRUMP, DONALD J.",REP,P
8,C00580100,N,Q2,P,201507159000184790,15,IND,"CARRERAS, DENNIS",COCONUT CREEK,FL,33073,"BERSTONE PAYMENT SYSTEMS, LLC",CEO,2015-06-17,250,NaN,SA17A.4199,1015464,NaN,NaN,4071620151247220753,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,P,"TRUMP, DONALD J.",REP,P
9,C00580100,N,Q2,P,201507159000184791,15,IND,"CARTER, SCOTT",LUBBOCK,TX,79407,OTTO'S GRANARY,MANAGER/OWNER,2015-06-19,250,NaN,SA17A.4514,1015464,NaN,NaN,4071620151247220754,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,P,"TRUMP, DONALD J.",REP,P


In [719]:
#remove bookkeeping variables 
keep = ['NAME', 'CITY', 'STATE', 'ZIP_CODE', 'EMPLOYER', 'OCCUPATION', 'TRANSACTION_DT', 'TRANSACTION_AMT', 
        'CMTE_TP', 'CAND_NAME', 'CAND_PTY_AFFILIATION', 'CMTE_NM',]
data = pd.DataFrame(dt_merged3, columns = keep) 

In [720]:
#number of contributions by candidate
data.groupby('CAND_NAME')['NAME'].count()

CAND_NAME
CLINTON, HILLARY RODHAM    406413
TRUMP, DONALD J.           15295 
Name: NAME, dtype: int64

## Check for Tech Names

In [721]:
tech_names = pd.read_csv('tech_names.csv')
#strip white space from column names
tech_names.columns = tech_names.columns.str.strip()
#strip white space from column values 
for col in tech_names.columns: 
    tech_names[col] = tech_names[col].str.strip().str.upper()
tech_names.head(3)

,First,Middle,Last,Suffix,Role,Company,Company2,Company3,Unnamed: 8
0,MARVIN,NaN,AMMORI,NaN,GENERAL COUNSEL,HYPERLOOP ONE,NaN,NaN,NaN
1,ADRIAN,NaN,AOUN,NaN,FOUNDER/CEO,FORWARD,NaN,NaN,NaN
2,GREG,NaN,BADROS,NaN,FOUNDER,PREPARED MIND INNOVATIONS,FACEBOOK,NaN,NaN


In [722]:
#split name in fec data into first and last
first_last = data['NAME'].str.strip().str.upper().str.split(',', expand=True)
first_last.describe()

,0,1,2,3
count,421708,421703,95,1
unique,56926,24688,44,1
top,SMITH,SUSAN,MD,DR. THOMAS E.
freq,2548,5904,12,1


In [723]:
#There are a small number of records where the last name is in the first name column. Test to see if contain any tech last names
#Due to the very small number of these records and because they are not applicable (not in list of tech). They will be ignored. 
results = []
for i in range(len(tech_names)):
    response = first_last[~first_last[2].isnull()][2].str.contains(tech_names.Last.values[i])
    results.append(response)
response.describe()

count    95   
mean     0    
std      0    
min      False
25%      0    
50%      0    
75%      0    
max      False
Name: 2, dtype: object

In [724]:
data.iloc[35726,:]

NAME                    KARP, DAVID            
CITY                    NEW YORK               
STATE                   NY                     
ZIP_CODE                100106212              
EMPLOYER                TUMBLR, INC.           
OCCUPATION              CEO                    
TRANSACTION_DT          2015-05-04 00:00:00    
TRANSACTION_AMT         2.7e+03                
CMTE_TP                 P                      
CAND_NAME               CLINTON, HILLARY RODHAM
CAND_PTY_AFFILIATION    DEM                    
CMTE_NM                 HILLARY FOR AMERICA    
Name: 35726, dtype: object

In [725]:
#check to find rows in data that both contain first letter of the first name and contain last name (*Note not 'match')
results = []
for i in range(len(tech_names)):
    response = first_last[0].str.contains(tech_names.Last[i]) & first_last[1].str.contains(tech_names.First[i])
    isTrue = response[response == True].index
    results.append([tech_names.Last[i],tech_names.First[i], tech_names.Company[i], isTrue.values])

In [726]:
results[0:13]

[['AMMORI', 'MARVIN', 'HYPERLOOP ONE', array([], dtype=int64)],
 ['AOUN', 'ADRIAN', 'FORWARD', array([], dtype=int64)],
 ['BADROS', 'GREG', 'PREPARED MIND INNOVATIONS', array([], dtype=int64)],
 ['BANKS', 'CLAYTON', 'SILICON HARLEM', array([], dtype=int64)],
 ['BARNES', 'PHIN', 'FIRST ROUND CAPITAL', array([], dtype=int64)],
 ['BASHAMBU', 'NITI', 'IAC APPLICATIONS', array([], dtype=int64)],
 ['BATTELLE', 'JOHN', 'NEWCO', array([], dtype=int64)],
 ['BDEIR', 'AYAH', 'LITTLE BITS', array([], dtype=int64)],
 ['BEIM', 'PIRAYE', 'CELMATIX', array([], dtype=int64)],
 ['BODNICK', 'MARC', 'ELEVATION PARTNERS', array([], dtype=int64)],
 ['BORTHWICK', 'JOHN', 'BETAWORKS', array([], dtype=int64)],
 ['BREZINA', 'MATT', 'SINCERELY', array([], dtype=int64)],
 ['BROWN-PHILPOT', 'STACY', 'TASKRABBIT', array([46310, 65105])]]

In [727]:
#convert results into a dataframe 
frames=[]
for row in results:
    frames.append(pd.DataFrame(row).T)
    search_results = pd.concat(frames).reset_index()

In [728]:
search_results.head(13)

,index,0,1,2,3
0,0,AMMORI,MARVIN,HYPERLOOP ONE,[]
1,0,AOUN,ADRIAN,FORWARD,[]
2,0,BADROS,GREG,PREPARED MIND INNOVATIONS,[]
3,0,BANKS,CLAYTON,SILICON HARLEM,[]
4,0,BARNES,PHIN,FIRST ROUND CAPITAL,[]
5,0,BASHAMBU,NITI,IAC APPLICATIONS,[]
6,0,BATTELLE,JOHN,NEWCO,[]
7,0,BDEIR,AYAH,LITTLE BITS,[]
8,0,BEIM,PIRAYE,CELMATIX,[]
9,0,BODNICK,MARC,ELEVATION PARTNERS,[]


In [729]:
data_rows = []
check_list_names = []
check_employers = []
for i in range(len(search_results)): 
    for list_item in search_results[3][i]: 
        response = data.iloc[list_item,:]
        check_name = str(search_results[0][i]) + ', ' + str(search_results[1][i])
        employer = search_results[2][i]
        data_rows.append(response)
        check_list_names.append(check_name)
        check_employers.append(employer)
        data_results = pd.DataFrame(data_rows)
data_results['tech_names'] = check_list_names
data_results['tech_employer'] = check_employers

In [730]:
data_results.columns.values

array(['NAME', 'CITY', 'STATE', 'ZIP_CODE', 'EMPLOYER', 'OCCUPATION',
       'TRANSACTION_DT', 'TRANSACTION_AMT', 'CMTE_TP', 'CAND_NAME',
       'CAND_PTY_AFFILIATION', 'CMTE_NM', 'tech_names', 'tech_employer'], dtype=object)

In [731]:
review = data_results[['tech_names', 'NAME', 'CITY', 'STATE', 'ZIP_CODE', 'tech_employer','EMPLOYER', 'OCCUPATION',
       'TRANSACTION_DT', 'TRANSACTION_AMT', 'CMTE_TP', 'CAND_NAME',
       'CAND_PTY_AFFILIATION', 'CMTE_NM', ]]

In [732]:
len(review)

132

In [733]:
#manually review and verify 
review.to_csv("tech_names_review.csv", dtypeencoding = 'utf-8')

In [743]:
#reimport reviewed list 
reviewed = pd.read_csv("tech_names_review.csv", encoding = 'utf-8', dtype = 'str')
reviewed.head(3)

,Unnamed: 0,tech_names,NAME,CITY,STATE,ZIP_CODE,tech_employer,EMPLOYER,accurate,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_TP,CAND_NAME,CAND_PTY_AFFILIATION,CMTE_NM
0,295159,"HARRIS, DONNA","HARRIS, DONNA",EMPIRE,AL,35063,1776,RETIRED,no,RETIRED,5/22/16,25,P,"TRUMP, DONALD J.",REP,"DONALD J. TRUMP FOR PRESIDENT, INC."
1,86557,"KEENAN, KIM","KEENAN, KIMBERLY",NAPLES,FL,34120,MULTICULTURAL MEDIA,COLLIER COUNTY PUBLIC SCHOOLS,no,TEACHER,3/9/16,250,P,"TRUMP, DONALD J.",REP,"DONALD J. TRUMP FOR PRESIDENT, INC."
2,293236,"LLOYD, MARK","LLOYD, MARK",NEW YORK,NY,10022,UNIVERSITY OF SOUTHERN CALIFORNIA - ANNENBERG SCHOOL,DONALD J TRUMP FOR PRESIDENT,no,STAFF,5/13/16,638,P,"TRUMP, DONALD J.",REP,"DONALD J. TRUMP FOR PRESIDENT, INC."


In [744]:
#remove white space and make upper 
for col in reviewed.columns: 
    reviewed[col] = reviewed[col].str.strip().str.upper()
reviewed.head(3)

,Unnamed: 0,tech_names,NAME,CITY,STATE,ZIP_CODE,tech_employer,EMPLOYER,accurate,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_TP,CAND_NAME,CAND_PTY_AFFILIATION,CMTE_NM
0,295159,"HARRIS, DONNA","HARRIS, DONNA",EMPIRE,AL,35063,1776,RETIRED,NO,RETIRED,5/22/16,25,P,"TRUMP, DONALD J.",REP,"DONALD J. TRUMP FOR PRESIDENT, INC."
1,86557,"KEENAN, KIM","KEENAN, KIMBERLY",NAPLES,FL,34120,MULTICULTURAL MEDIA,COLLIER COUNTY PUBLIC SCHOOLS,NO,TEACHER,3/9/16,250,P,"TRUMP, DONALD J.",REP,"DONALD J. TRUMP FOR PRESIDENT, INC."
2,293236,"LLOYD, MARK","LLOYD, MARK",NEW YORK,NY,10022,UNIVERSITY OF SOUTHERN CALIFORNIA - ANNENBERG SCHOOL,DONALD J TRUMP FOR PRESIDENT,NO,STAFF,5/13/16,638,P,"TRUMP, DONALD J.",REP,"DONALD J. TRUMP FOR PRESIDENT, INC."


In [745]:
reviewed['TRANSACTION_AMT'] = reviewed['TRANSACTION_AMT'].astype('float')
reviewed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 16 columns):
Unnamed: 0              132 non-null object
tech_names              132 non-null object
NAME                    132 non-null object
CITY                    132 non-null object
STATE                   132 non-null object
ZIP_CODE                132 non-null object
tech_employer           129 non-null object
EMPLOYER                112 non-null object
accurate                133 non-null object
OCCUPATION              130 non-null object
TRANSACTION_DT          132 non-null object
TRANSACTION_AMT         132 non-null float64
CMTE_TP                 132 non-null object
CAND_NAME               132 non-null object
CAND_PTY_AFFILIATION    132 non-null object
CMTE_NM                 132 non-null object
dtypes: float64(1), object(15)
memory usage: 136.0+ MB


In [746]:
tech_givers = reviewed[reviewed['accurate'] == 'YES']

In [747]:
len(tech_givers)

59

In [748]:
float(len(tech_givers))/float(len(tech_names))

0.4068965517241379

In [749]:
tech_givers

,Unnamed: 0,tech_names,NAME,CITY,STATE,ZIP_CODE,tech_employer,EMPLOYER,accurate,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_TP,CAND_NAME,CAND_PTY_AFFILIATION,CMTE_NM
73,46310,"BROWN-PHILPOT, STACY","BROWN-PHILPOT, STACY",PALO ALTO,CA,943061426,TASKRABBIT,TASKRABBIT,YES,COO,9/19/15,500,P,"CLINTON, HILLARY RODHAM",DEM,HILLARY FOR AMERICA
74,65105,"BROWN-PHILPOT, STACY","BROWN-PHILPOT, STACY",PALO ALTO,CA,943061426,TASKRABBIT,TASKRABBIT,YES,COO,10/28/15,2200,P,"CLINTON, HILLARY RODHAM",DEM,HILLARY FOR AMERICA
75,37543,"BURNHAM, BRAD","BURNHAM, BRADFORD",NEW YORK,NY,100118843,UNION SQUARE VENTURES,UNION SQUARE VENTURES,YES,BUSINESS,6/30/15,2700,P,"CLINTON, HILLARY RODHAM",DEM,HILLARY FOR AMERICA
76,6094,"CARTER, TROY","CARTER, TROY",CULVER CITY,CA,902323149,ATOM FACTORY,ATOM FACTORY,YES,MEDIA EXECUTIVE,6/3/15,2700,P,"CLINTON, HILLARY RODHAM",DEM,HILLARY FOR AMERICA
77,8853,"CASSIDY, SUKHINDER","SINGH CASSIDY, SUKHINDER",ATHERTON,CA,940272230,JOYUS,JOYUS,YES,CEO,6/11/15,2700,P,"CLINTON, HILLARY RODHAM",DEM,HILLARY FOR AMERICA
78,2738,"CERF, VINT","CERF, VINTON PH.D.",MCLEAN,VA,221022234,NaN,GOOGLE,YES,VICE PRESIDENT,4/24/15,2700,P,"CLINTON, HILLARY RODHAM",DEM,HILLARY FOR AMERICA
79,38512,"CERF, VINT","CERF, VINTON PH.D.",MCLEAN,VA,221022234,NaN,GOOGLE,YES,VICE PRESIDENT,7/8/15,250,P,"CLINTON, HILLARY RODHAM",DEM,HILLARY FOR AMERICA
80,73912,"CERF, VINT","CERF, VINTON PH.D.",MCLEAN,VA,221022234,NaN,GOOGLE,YES,VICE PRESIDENT,10/23/15,250,P,"CLINTON, HILLARY RODHAM",DEM,HILLARY FOR AMERICA
81,29386,"CHOPRA, ANEESH","CHOPRA, ANEESH",ARLINGTON,VA,222074122,NAVHEALTH,HUNCH ANALYTICS,YES,TECHNOLOGY,6/24/15,2700,P,"CLINTON, HILLARY RODHAM",DEM,HILLARY FOR AMERICA
82,25731,"COHEN, TOD","COHEN, TOD",PALO ALTO,CA,943013444,STUBHUB,EBAY INC.,YES,ATTORNEY,4/13/15,2700,P,"CLINTON, HILLARY RODHAM",DEM,HILLARY FOR AMERICA


In [750]:
tech_givers.groupby(by = 'tech_names')['TRANSACTION_AMT'].sum().reset_index()

,tech_names,TRANSACTION_AMT
0,"BROWN-PHILPOT, STACY",2700
1,"BURNHAM, BRAD",2700
2,"CARTER, TROY",2700
3,"CASSIDY, SUKHINDER",2700
4,"CERF, VINT",3200
5,"CHOPRA, ANEESH",2700
6,"COHEN, TOD",2725
7,"DILLER, BARRY",2700
8,"DYSON, ESTHER",2700
9,"ERRETT, AMY",2700


In [751]:
tech_names["tech_names"] = tech_names.Last.str.cat(tech_names.First, sep = ', ')
tech_names["tech_names"].head(3)

0    AMMORI, MARVIN
1    AOUN, ADRIAN  
2    BADROS, GREG  
Name: tech_names, dtype: object

In [752]:
tech_givers_table = pd.merge(tech_givers, tech_names, on = 'tech_names', how = 'right')
tech_givers_table.tail(3)

,Unnamed: 0,tech_names,NAME,CITY,STATE,ZIP_CODE,tech_employer,EMPLOYER,accurate,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_TP,CAND_NAME,CAND_PTY_AFFILIATION,CMTE_NM,First,Middle,Last,Suffix,Role,Company,Company2,Company3,Unnamed: 8
154,NaN,"WU, TIM",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TIM,NaN,WU,NaN,PROFESSOR OF LAW,COLUMBIA UNIVERSITY,NaN,NaN,NaN
155,NaN,"YANG, ANDREW",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANDREW,NaN,YANG,NaN,FOUNDER/CEO,VENTURE FOR AMERICA,NaN,NaN,NaN
156,NaN,"ZUCKERBERG, ARIELLE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARIELLE,NaN,ZUCKERBERG,NaN,PARTNER,KLEINER PERKINS CAUFIELD & BYERS,NaN,NaN,NaN


In [753]:
tech_givers_table.columns.values[:]

array(['Unnamed: 0', u'tech_names', u'NAME', u'CITY', u'STATE',
       u'ZIP_CODE', u'tech_employer', u'EMPLOYER', u'accurate',
       u'OCCUPATION', u'TRANSACTION_DT', u'TRANSACTION_AMT', u'CMTE_TP',
       u'CAND_NAME', u'CAND_PTY_AFFILIATION', u'CMTE_NM', 'First',
       'Middle', 'Last', 'Suffix', 'Role', 'Company', 'Company2',
       'Company3', 'Unnamed: 8'], dtype=object)

In [754]:
tech_givers_table = tech_givers_table[['Unnamed: 0', u'tech_names', u'NAME', u'CITY', u'STATE', u'ZIP_CODE', u'tech_employer', u'EMPLOYER', u'accurate', u'OCCUPATION', u'TRANSACTION_DT', u'TRANSACTION_AMT', u'CMTE_TP',u'CAND_NAME', u'CAND_PTY_AFFILIATION', u'CMTE_NM', 'Role', 'Company', 'Company2','Company3']]

In [755]:
tech_givers_table.to_csv('tech_givers_final.csv', encoding = 'utf-8')

## Individual Donations to Presidential Candidates by Occupation

In [756]:
#limit data to 2016 presidential race
individual = data
individual.head(3)

,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_TP,CAND_NAME,CAND_PTY_AFFILIATION,CMTE_NM
0,"ANDALORO, JIM",MARBLEHEAD,MA,01945,"METAL TRONICS, INC.",OWNER,2015-06-30,500,P,"TRUMP, DONALD J.",REP,"DONALD J. TRUMP FOR PRESIDENT, INC."
1,"ANTON, FRANCINE",SPRING LAKE,MI,49456,RETIRED,RETIRED,2015-06-24,250,P,"TRUMP, DONALD J.",REP,"DONALD J. TRUMP FOR PRESIDENT, INC."
2,"ATKINS, LORI",SAN ANTONIO,TX,78231,TEXAS SINUS CENTER,PHYSICIAN ASSISTANT,2015-06-20,250,P,"TRUMP, DONALD J.",REP,"DONALD J. TRUMP FOR PRESIDENT, INC."


### By Occupation

In [765]:
by_occ = individual.groupby(['CAND_NAME','OCCUPATION'],  as_index=False)['TRANSACTION_AMT'].sum()

In [766]:
by_occ = by_occ.sort_values(by =['CAND_NAME', 'TRANSACTION_AMT', 'OCCUPATION'], ascending=False)

In [767]:
by_occ.to_csv('donations_by_occ.csv')
len(by_occ)

22333

In [768]:
#Number of occupations
occ_types = pd.unique(by_occ.OCCUPATION.ravel())     
len(occ_types)

21362

In [769]:
#number of candidates receiving support from each occupation 
occ_type_freq = by_occ.OCCUPATION.value_counts()
occ_type_freq.to_csv('occupation_types.csv')


In [827]:
job = ('YOGA TEACHER')
job = by_occ[by_occ['OCCUPATION'].str.contains(job)]
job = job.sort_values(by = 'TRANSACTION_AMT', ascending=False)
print job.groupby('CAND_NAME')['TRANSACTION_AMT'].sum().reset_index()
print job.groupby('CAND_NAME')['OCCUPATION'].value_counts().reset_index()

                 CAND_NAME  TRANSACTION_AMT
0  CLINTON, HILLARY RODHAM  24390          
                 CAND_NAME                    OCCUPATION  0
0  CLINTON, HILLARY RODHAM  KIDS YOGA TEACHER             1
1  CLINTON, HILLARY RODHAM  THERAPEUTIC YOGA TEACHER      1
2  CLINTON, HILLARY RODHAM  YOGA TEACHER                  1
3  CLINTON, HILLARY RODHAM  YOGA TEACHER/PSYCHOTHERAPIST  1


In [777]:
job = ('PSYCHIC')
job = by_occ[by_occ['OCCUPATION'].str.contains(job)]
job.sort_values(by = 'TRANSACTION_AMT', ascending=False)

,CAND_NAME,OCCUPATION,TRANSACTION_AMT
22046,"TRUMP, DONALD J.",SPIRITUAL PSYCHIC SOUL THERAPIST,500
14099,"CLINTON, HILLARY RODHAM",PSYCHIC,20


In [784]:
individual[individual['OCCUPATION'] == 'SPIRITUAL PSYCHIC SOUL THERAPIST']

,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_TP,CAND_NAME,CAND_PTY_AFFILIATION,CMTE_NM
78312,"AMBER, SHARMAI",PAAUILO,HI,96776,SELF EMPLOYED,SPIRITUAL PSYCHIC SOUL THERAPIST,2015-08-01,500,P,"TRUMP, DONALD J.",REP,"DONALD J. TRUMP FOR PRESIDENT, INC."


In [828]:
job = ('FUNERAL')
job = by_occ[by_occ['OCCUPATION'].str.contains(job)]
job = job.sort_values(by = 'TRANSACTION_AMT', ascending=False)
print job.groupby('CAND_NAME')['TRANSACTION_AMT'].sum().reset_index()
print job.groupby('CAND_NAME')['OCCUPATION'].value_counts().reset_index()

                 CAND_NAME  TRANSACTION_AMT
0  CLINTON, HILLARY RODHAM  16350          
1  TRUMP, DONALD J.         1386           
                 CAND_NAME                            OCCUPATION  0
0  CLINTON, HILLARY RODHAM  FUNERAL DIRECTOR                      1
1  CLINTON, HILLARY RODHAM  FUNERAL DIRECTOR, OWNER               1
2  CLINTON, HILLARY RODHAM  FUNERAL DIRECTOR/EMBALMER             1
3  CLINTON, HILLARY RODHAM  FUNERAL SERVICE                       1
4  CLINTON, HILLARY RODHAM  FUNERAL SERVICES                      1
5  CLINTON, HILLARY RODHAM  MEDICAL ONCOLOGIST, FUNERAL DIRECTOR  1
6  CLINTON, HILLARY RODHAM  OWNER, FUNERAL DIRECTOR               1
7  TRUMP, DONALD J.         FUNERAL DIRECTOR                      1
8  TRUMP, DONALD J.         FUNERAL SERVICE PROVIDER              1


In [834]:
job = ('NOT EMPLOYED')
job = by_occ[by_occ['OCCUPATION'].str.contains(job)]
job = job.sort_values(by = 'TRANSACTION_AMT', ascending=False)
print job.groupby('CAND_NAME')['TRANSACTION_AMT'].sum().reset_index()
print job.groupby('CAND_NAME').count().reset_index()

                 CAND_NAME  TRANSACTION_AMT
0  CLINTON, HILLARY RODHAM  1877656        
1  TRUMP, DONALD J.         2815           
                 CAND_NAME  OCCUPATION  TRANSACTION_AMT
0  CLINTON, HILLARY RODHAM  1           1              
1  TRUMP, DONALD J.         1           1              


In [835]:
job = ('UNEMPLOYED')
job = by_occ[by_occ['OCCUPATION'].str.contains(job)]
job.sort_values(by = 'TRANSACTION_AMT', ascending=False)

,CAND_NAME,OCCUPATION,TRANSACTION_AMT
22224,"TRUMP, DONALD J.",UNEMPLOYED,6240
18638,"CLINTON, HILLARY RODHAM",UNEMPLOYED,25


In [836]:
job = ('DATA')
job = by_occ[by_occ['OCCUPATION'].str.contains(job)]
job = job.sort_values(by = 'TRANSACTION_AMT', ascending=False)
print job.groupby('CAND_NAME')['TRANSACTION_AMT'].sum().reset_index()
print job.groupby('CAND_NAME')['OCCUPATION'].count().reset_index()

                 CAND_NAME  TRANSACTION_AMT
0  CLINTON, HILLARY RODHAM  89383          
1  TRUMP, DONALD J.         2282           
                 CAND_NAME  OCCUPATION
0  CLINTON, HILLARY RODHAM  97        
1  TRUMP, DONALD J.         6         


In [837]:
job = ('PILOT')
job = by_occ[by_occ['OCCUPATION'].str.contains(job)]
job.sort_values(by = 'TRANSACTION_AMT', ascending=False)
job = job.groupby('CAND_NAME')['OCCUPATION'].value_counts().reset_index()
job.columns=('CAND_NAME', 'OCCUPATION', 'FREQUENCY')
job = job.groupby('CAND_NAME').sum()
job

,FREQUENCY
CAND_NAME,
"CLINTON, HILLARY RODHAM",19
"TRUMP, DONALD J.",6


In [838]:
job = ('HOTEL')
job = by_occ[by_occ['OCCUPATION'].str.contains(job)]
job = job.groupby('CAND_NAME')['OCCUPATION'].value_counts().reset_index()
job.columns=('CAND_NAME', 'OCCUPATION', 'FREQUENCY')
job = job.groupby('CAND_NAME').sum()
job

,FREQUENCY
CAND_NAME,
"CLINTON, HILLARY RODHAM",32
"TRUMP, DONALD J.",10


In [839]:
job = ('INVESTMENT')
job = by_occ[by_occ['OCCUPATION'].str.contains(job)==True]
job = job.groupby('CAND_NAME')['OCCUPATION'].value_counts().reset_index()
job.columns=('CAND_NAME', 'OCCUPATION', 'FREQUENCY')
job = job.groupby('CAND_NAME').sum()
job

,FREQUENCY
CAND_NAME,
"CLINTON, HILLARY RODHAM",119
"TRUMP, DONALD J.",23


In [840]:
job = ('HOMEMAKER')
job = by_occ[by_occ['OCCUPATION'].str.contains(job)]
job.sort_values(by = 'TRANSACTION_AMT', ascending=False)
job = job.groupby('CAND_NAME')['OCCUPATION'].value_counts().reset_index()
job.columns=('CAND_NAME', 'OCCUPATION', 'FREQUENCY')
job = job.groupby('CAND_NAME').sum()
job

,FREQUENCY
CAND_NAME,
"CLINTON, HILLARY RODHAM",57
"TRUMP, DONALD J.",1


In [841]:
job = ('RETIRED')
job = by_occ[by_occ['OCCUPATION'].str.contains(job)]
job.sort_values(by = 'TRANSACTION_AMT', ascending=False)
job.sort_values(by = 'TRANSACTION_AMT', ascending=False)
job = job.groupby('CAND_NAME')['OCCUPATION'].value_counts().reset_index()
job.columns=('CAND_NAME', 'OCCUPATION', 'FREQUENCY')
job = job.groupby('CAND_NAME').sum()
job

,FREQUENCY
CAND_NAME,
"CLINTON, HILLARY RODHAM",392
"TRUMP, DONALD J.",5


## Select Occupations

In [842]:
occ_freq = individual.groupby('CAND_NAME')['OCCUPATION'].value_counts().reset_index()
occ_freq.columns = ('CAND_NAME', 'OCCUPATION', 'FREQUENCY' )
occ_freq

,CAND_NAME,OCCUPATION,FREQUENCY
0,"CLINTON, HILLARY RODHAM",RETIRED,85495
1,"CLINTON, HILLARY RODHAM",ATTORNEY,22531
2,"CLINTON, HILLARY RODHAM",HOMEMAKER,8523
3,"CLINTON, HILLARY RODHAM",TEACHER,7895
4,"CLINTON, HILLARY RODHAM",PHYSICIAN,7861
5,"CLINTON, HILLARY RODHAM",CONSULTANT,7856
6,"CLINTON, HILLARY RODHAM",PROFESSOR,7672
7,"CLINTON, HILLARY RODHAM",NOT EMPLOYED,6780
8,"CLINTON, HILLARY RODHAM",LAWYER,6523
9,"CLINTON, HILLARY RODHAM",WRITER,3414


In [843]:
occ_freq_candidate = occ_freq.groupby('CAND_NAME')['FREQUENCY'].describe()
occ_freq_candidate

CAND_NAME                     
CLINTON, HILLARY RODHAM  count    20007.00
                         mean     19.67   
                         std      646.83  
                         min      1.00    
                         25%      1.00    
                         50%      2.00    
                         75%      5.00    
                         max      85495.00
TRUMP, DONALD J.         count    2326.00 
                         mean     6.35    
                         std      100.90  
                         min      1.00    
                         25%      1.00    
                         50%      1.00    
                         75%      2.00    
                         max      4768.00 
dtype: float64

In [29]:
#Top 3 that donate to all candidates across the board
#Top 3 by total aggregate amount donated 
#Top 3 by count/frequency by candidate  
#Top 3 that employ the most people (per fed stats)
#3 at random/ fun (Yoga, Test Pilot, etc.)


SyntaxError: invalid syntax (<ipython-input-29-dc8c446cedb1>, line 1)